## 引用Keras

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, Conv2D, MaxPooling2D


Using TensorFlow backend.


## 讀取MNIST 數據

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
x_test.shape

(10000, 28, 28)

In [5]:
x_train = x_train.reshape(60000, 28,28,1)
x_test  = x_test.reshape(10000, 28,28,1)
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')
x_train /= 255
x_test  /= 255

In [6]:
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test  = keras.utils.to_categorical(y_test, n_classes)

## 設定網路參數

In [7]:
n_hidden_1 = 256 
n_classes  = 10 

training_epochs = 15
batch_size = 100

## 建立卷積神經網路

In [8]:
model = keras.Sequential()
model.add(Conv2D(
            filters=16,
            kernel_size = (3,3),
            padding = 'same',
            input_shape = (28,28,1), 
            activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(n_hidden_1, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


## 訓練神經網路

In [10]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=15,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 11s 189us/step - loss: 0.2057 - acc: 0.9396 - val_loss: 0.0887 - val_acc: 0.9722
Epoch 2/15
60000/60000 [==============================] - 11s 184us/step - loss: 0.0644 - acc: 0.9809 - val_loss: 0.0544 - val_acc: 0.9829
Epoch 3/15
60000/60000 [==============================] - 12s 194us/step - loss: 0.0429 - acc: 0.9869 - val_loss: 0.0484 - val_acc: 0.9836
Epoch 4/15
60000/60000 [==============================] - 12s 197us/step - loss: 0.0298 - acc: 0.9907 - val_loss: 0.0441 - val_acc: 0.9858
Epoch 5/15
60000/60000 [==============================] - 12s 199us/step - loss: 0.0218 - acc: 0.9931 - val_loss: 0.0460 - val_acc: 0.9851
Epoch 6/15
60000/60000 [==============================] - 12s 204us/step - loss: 0.0157 - acc: 0.9951 - val_loss: 0.0486 - val_acc: 0.9844
Epoch 7/15
60000/60000 [==============================] - 12s 199us/step - loss: 0.0113 - acc: 0.9968 - val_loss: 0.

## 辨識數字

In [11]:
from PIL import Image
import numpy
img = Image.open('/Users/davidchiu/Desktop/4.jpg').convert('L')
img  = numpy.asarray(img) 

In [13]:
img2 = (255 - img) / 255 

In [15]:
img3 = img2.reshape(28,28,1)

In [16]:
y = numpy.array([img3])

In [17]:
numpy.argmax(model.predict(y))

4